In [ ]:
#Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
from collections import defaultdict

In [ ]:
#Use one ./ if running from run_notebooks.py
#Use two ../if running directly from Jupyter Notebooks
file_beg = './NHANES-Downloader/data/csv_data/'

In [ ]:
#Import all the files
files1 = glob.glob(file_beg+'1999-2000/*/*.csv')
files2 = glob.glob(file_beg+'2001-2002/*/*.csv')
files3 = glob.glob(file_beg+'2003-2004/*/*.csv')
files4 = glob.glob(file_beg+'2005-2006/*/*.csv')
files5 = glob.glob(file_beg+'2007-2008/*/*.csv')
files6 = glob.glob(file_beg+'2009-2010/*/*.csv')
files7 = glob.glob(file_beg+'2011-2012/*/*.csv')
files8 = glob.glob(file_beg+'2013-2014/*/*.csv')
files9 = glob.glob(file_beg+'2015-2016/*/*.csv')

In [ ]:
#Add files into a list
file_list = [files1, files2, files3, files4, files5, 
          files6, files7, files8, files9]

In [ ]:
#Add files into a list of sorted and dictionaries
file_list_dict = []
for x in file_list:
    x.sort()
    file_list_dict.append(dict(enumerate(x)))

In [ ]:
alq_indx = [73, 86, 100, 92, 88, 94, 88, 109, 62]

In [ ]:
dfs = defaultdict(int)
for i in range(0,len(alq_indx)):
    dfs[i] = pd.read_csv(file_list_dict[i][alq_indx[i]])

## Functions: Recategorize values, Count Values, Drop Rows

In [ ]:
#Recategorize function
def recategorize(df, name, replace_dict):
    df[name].replace(
    to_replace=replace_dict,
    inplace=True
)
    
#Count values function
def count_vals(df, name):
    df_count = df.groupby(name)['SEQN'].nunique()
    print(df_count,"\n\n","NaN: ", df[name].isnull().sum())
    
#Drop rows that include certain values
def drop_rows(df, name, val_list):
    df.drop(df[df[name].isin(val_list)].index, inplace=True)

## Important Variables

In [ ]:
#1999-2016
var_names = ["SEQN", "ALQ101"]

In [ ]:
#To display all columns in Jupyter Notebooks
#pd.set_option('display.max_columns', 500)

## Rename 1999-2000 and 2001-2002 columns

In [ ]:
#Rename HUD070 to HUQ070
dfs[0].rename(columns={'ALQ100': 'ALQ101'}, inplace=True)
#Rename HUD070 to HUQ070
dfs[1].rename(columns={'ALD100': 'ALQ101'}, inplace=True)

## Make a copy of the dataframes

In [ ]:
#1999-20016
df0 = dfs[0][var_names].copy() #1999-2000
df1 = dfs[1][var_names].copy() #2001-2002
df2 = dfs[2][var_names].copy() #2003-2004
df3 = dfs[3][var_names].copy() #2005-2006
df4 = dfs[4][var_names].copy() #2007-2008
df5 = dfs[5][var_names].copy() #2009-2010
df6 = dfs[6][var_names].copy() #2011-2012
df7 = dfs[7][var_names].copy() #2013-2014
df8 = dfs[8][var_names].copy() #2015-2016

## Recategorize values

In [ ]:
#No recategorizing needed

## Rename columns 1999 - 2016

In [ ]:
#No renaming needed

## Append years 1999 - 2016

In [ ]:
years = ["1999-2000","2001-2002","2003-2004","2005-2006","2007-2008", 
        "2009-2010","2011-2012","2013-2014","2015-2016"]

In [ ]:
frames = [df0, df1, df2, df3, df4, df5, df6, df7, df8]

In [ ]:
#Add years as a column
for i, df in enumerate(frames):
    df["Year"] = years[i]

In [ ]:
result_1999_2016 = pd.concat(frames, keys = years)

In [ ]:
result_1999_2016_cleaned = result_1999_2016.copy()

In [ ]:
result_1999_2016_cleaned

## Count values for each column

In [ ]:
col_names = var_names

In [ ]:
len(col_names)

In [ ]:
#ALQ - Ever told you had high blood pressure
count_vals(result_1999_2016, col_names[1])

In [ ]:
len(result_1999_2016)

## Remove missing values:

In [ ]:
drop_rows(result_1999_2016_cleaned, col_names[1], [np.nan, 7, 9])

## See if missing values have been correctly removed:

In [ ]:
count_vals(result_1999_2016_cleaned, col_names[1])

In [ ]:
before = len(result_1999_2016)
before

In [ ]:
after = len(result_1999_2016_cleaned)
after

In [ ]:
(before-after)/after

## MongoDB Insertion

In [ ]:
#Import MongoClient
from pymongo import MongoClient

#Create a MongoClient to run the MongoDB instance
client = MongoClient("localhost", 27017)

In [ ]:
#Connect to existing database
db = client.NHANES
db

In [ ]:
db.list_collection_names()

In [ ]:
#Creating a collection
alq = db.alq

In [ ]:
#If collections exist, then drop
if 'alq' in db.list_collection_names():
    alq.drop()
    db.list_collection_names()

## Create new collection to input into database

In [ ]:
result_1999_2016_cleaned[:3]

In [ ]:
#Set SEQN as _id (Primary Key)
result_1999_2016_cleaned.rename(columns= {'SEQN':'_id'}, inplace=True)

In [ ]:
#Dataframe to dictionary
alq_dict = result_1999_2016_cleaned.to_dict(orient='records')

In [ ]:
alq_dict[0]

In [ ]:
#Insert collection
alq.insert_many(alq_dict)

In [ ]:
db.list_collection_names()